# Group members
Badil Mujovi (274632)\
Aubin Mercier (316260)\
Mehmet Furkan Dogan (370234)\
Andrea Grillo (371099)

## Introduction

This project aims to combine vision, global and local navigation, and filtering to maneuver a Thymio robot in a predetermined environment.

Our environment consist of a white floor delimited by ArUco markers in the 4 corners. There is an ArUco marker on the Thymio and the goal is an ArUco marker too. Static obstacles are black polygones printed on the map. A camera is placed on top of the environment, to build a map of the static obstacles and compute the optimal path towards the goal.

The Thymio is placed at a random position, the camera tracks its position and orientation, and it has to reach the goal following the optimal path. The system is robust to 3D dynamic obstacles placed along the path, as well as camera covering for short periods of time and kidnapping, that is moving the robot to another place during its activity.

Our implementation includes the following features:
- Computer Vision for detecting obstacles and building the map
- Computer Vision for the tracking of the pose of the Thymio
- Global navigation computes the optimal path from the map using a visibility graph
- Local navigation makes the Thymio follow the computed path as a list of waypoints
- Obstacle avoidance by sensing the environment with proximity sensors
- Extended Kalman Filter allows pose estimation when the camera is covered using the fusion of data from camera and odometry
- Resilience to kidnapping by recomputing the global path when the camera detects that the position of the Thymio has changed
- ...


The main code of the system is implemented in this Jupyter noteboook. The submodules of the system have been implemented in Python classes, each one in a separate file. They are the following:
- Computer Vision - vision_thymio.py
- Global Navigation - global_navigation.py
- Extended Kalman Filter - extended_kalman_filter.py
- Local Navigation - local_navigation.py


Some scripts for testing the subsystems have been written, and they are located under the 'test' directory.
Moreover, for the estimation for the variance matrices for the Kalman Filter, measurements and subsequent data analysis in Matlab have been conducted. All the files related to this part can be found in the 'Sensor Noise Measurement' directory.

## 1. Computer Vision

TODO

I have found this picture, don't know if it makes sense to use it
<div><img src = "images\detectAruco.png" width = 500 height = 500></div>

## 2. Path Planning

TODO

## 3. Filtering

## 4. Local Navigation

The goal of the Local Navigation submodule is to make the Thymio follow the path computed by the Global Navigation part, while making the Thymio avoid dynamic obstacles that can be placed along the path.

The path is given as a list of tuples (x,y) representing the waypoints of the path. The path is initialized using the `define_path` function. In this function, also the `waypoint_counter` variable is initialized. This variable keeps track of which waypoint we are aiming for at the moment.

<div><b>Control routine</b></div>
The function which is called at every loop iteration is `control`, whose role is to decide whether to apply the controller to follow the path or the local avoidance routine.
The system starts in the `path_follow` modality, which is changed to `obstacle_avoidance` whenever the proximity sensor detect an obstacle. When no obstacle is detected anymore, the obstacle avoidance routine will be run for the following 5 loop cycles, to ensure that the obstacle is really been overcome and that the Thymio is far enough from it.

<br><div><b>Obstacle avoidance routine</b></div>



<br><div><b>Path follow routine</b></div>
The function `path_follow` has the pose of the Thymio as the only parameter, as the path is defined as an attribute of the Local Navigation class.
The idea is that the two variables to control are the linear and angular velocities, respectively $ v $ and $ \omega $.

The controller chosen is a simple proportional controller for the $ \omega $, and a predefined fixed velocity for $ v $. 

The control law is therefore:
$$ v = v_{fixed} $$
$$ \omega = K_p ( \theta - \gamma) $$
where $ \theta $ is the orientation of the Thymio and $ \gamma $ is the angle of the vector connecting the Thymio to the waypoint. 

### TODO INSERT IMAGE



The gain value $ k_p $ and the fixed velocity $ v_{fixed} have been tuned empiricaly by trial and error to obtain the best performance possible.

<div><img src = "images\controller.jpg" width = 500></div>



## Complete code

## Conclusion

# SCAFFOLDING

### Installing dependencies

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pyvisgraph
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install opencv-contrib-python
!{sys.executable} -m pip install tqdm

^C


### Constant Definition, libraries and submodules import

In [1]:
#  - - - CONSTANT DEFINITION - - -

USE_THYMIO = True

# sampling time
DT = .15

print("Constants defined")

#  - - - IMPORTS - - - 
import tdmclient.notebook
import time
import numpy as np

import local_navigation
import extended_kalman_filter
#from vision_thymio import *
import vision_thymio

print("Import executed successfully")

if USE_THYMIO:
    await tdmclient.notebook.start()
    print("Successfully connected to Thymio.")

Constants defined


c:\Users\andrea\Dropbox (Politecnico Di Torino Studenti)\Tirocinio Andrea Grillo\08 FIRMWARE\sensoriPaquitop\mobile-robotics-project\extended_kalman_filter.py:5: RuntimeWarning: invalid value encountered in multiply
  CAMERA_COVERED_COV = np.eye(3) * np.inf


Import executed successfully
Successfully connected to Thymio.


In [21]:
await tdmclient.notebook.stop()


In [28]:
# - - - RELOADING OF IMPORTED MODULES - - -
import importlib
importlib.reload(local_navigation)
#importlib.reload(vision_thymio)
#importlib.reload(extended_kalman_filter)

<module 'local_navigation' from 'c:\\Users\\andrea\\Dropbox (Politecnico Di Torino Studenti)\\Tirocinio Andrea Grillo\\08 FIRMWARE\\sensoriPaquitop\\mobile-robotics-project\\local_navigation.py'>

### Classes instantiation, helper functions to interact with the Thymio

In [2]:
# ISTANTIATE OBJECTS OF CLASSES

vis = vision_thymio.Vision_Thymio()
local_nav = local_navigation.Local_Navigation()
ekf = extended_kalman_filter.ExtendedKalmanFilter()

print("Classes defined correctly.")


# FUNCTION FOR THYMIO INTERACTION DEFINITION

@tdmclient.notebook.sync_var
def motor_go(left,right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    
@tdmclient.notebook.sync_var
def motor_stop():
    global motor_left_target,motor_right_target
    motor_left_target = 0
    motor_right_target = 0
    
@tdmclient.notebook.sync_var
def sensor_data():
    global prox_horizontal
    return prox_horizontal.copy()

@tdmclient.notebook.sync_var
def odometry():
    global motor_left_speed, motor_right_speed
    return (motor_left_speed, motor_right_speed)

print("Thymio functions defined correctly.")

Init Vision Thymio
Init Local Navigation
Init Extended Kalman Filter
Classes defined correctly.
Thymio functions defined correctly.


In [29]:
local_nav = local_navigation.Local_Navigation()


Init Local Navigation


In [3]:
# TEST THAT THE THYMIO IS LISTENING TO US
motor_go(500,500)
time.sleep(1)
motor_stop()

### Perspective and scaling of the image, static obstacle detection

In [4]:
# init of the perspective and scaling of the image
vis.getPerspectiveAndScaling()

# get obstacles on the map
img = vis.getCorrectedImage()
staticObstacleList = vis.getVisibilityGraph(img).copy()
print(staticObstacleList)


[]


### Detection of the Thymio and the goal, optimal path computation

In [39]:
try:
    foundGoal = foundThymio = False
    goalPos = []
    thymioPos = []

    # try ten times to get thymio and goal position on the map, if not print error and exit
    for i in range(10):
        img = vis.getCorrectedImage()
        ids,corners = vis.detectArucoMarkers(img)

        if not foundGoal:
            foundGoal, goalPos = vis.getGoalPosition(ids, corners)
        if not foundThymio:
            foundThymio, thymioPos = vis.getThymioPos(ids,corners) 

        if foundGoal and foundThymio:
            break
        
        time.sleep(0.1)
    if not (foundGoal and foundThymio):
        raise Exception("cannot find thymio or goal")

    print(f"Found thymio and goal at iteration {i}")
    print(f"Position of the thymio: ", thymioPos[0][0]/vis.scalingFactor, thymioPos[0][1]/vis.scalingFactor, thymioPos[1])
    print(f"Position of the goal: {goalPos/vis.scalingFactor}")


    path = vis.getOptimalPath(thymioPos[0], staticObstacleList.copy(), goalPos)

    # path reshaping to be used by the local navigation module
    path = list(map(tuple, path.reshape((path.shape[0], 2))))
    
    print("Computed optimal path: ", path)

    local_nav.define_path(path)

    #EKF state initialization
    initial_state = np.array([thymioPos[0][0],thymioPos[0][1], thymioPos[1], 0, 0])  # [x, y, theta, v, w]
    ekf.state_initialization(initial_state)


except Exception as e:
    print("ERROR - cannot find goal or thymio")
    print("foundThymio", foundThymio)
    print("foundGoal", foundGoal)

motor_stop()


Unchecked Raise(exc=Call(func=Name(id='Exception', ctx=Load()), args=[Constant(value='cannot find thymio or goal')], keywords=[]))
Found thymio and goal at iteration 1
Position of the thymio:  371.0 509.0 2.0701430484750265
Position of the goal: [377. 622.]
Computed optimal path:  [(14.37625, 19.72375), (14.60875, 24.1025)]


In [40]:
finished = False
while not finished:
    # Getting the position from the camera
    img = vis.getCorrectedImage()
    ids,corners = vis.detectArucoMarkers(img)
    foundThymio, thymioPos = vis.getThymioPos(ids,corners) 
    
    if foundThymio:
        ekf.set_mode(extended_kalman_filter.NORMAL)

    else:
        print("- - - CAMERA COVERED - - -")
        ekf.set_mode(extended_kalman_filter.KIDNAPPED)
        continue

        # If there is no camera input


    x,y, theta = thymioPos[0][0], thymioPos[0][1], thymioPos[1] # Position and orientation from the camera
    
    #control_input = np.array([v,w])
    #measurement = np.array([x, y, theta, v, w])

    # Perform prediction and update steps
    #ekf.predict(control_input, DT)
    #ekf.update(measurement)

    #x, y, theta, v, w = ekf.state
    print(x,y,theta)
    sensors = sensor_data()

    print("-------------------------------------", sensors)
    
    (wl,wr),finished = local_nav.control((x,y,theta), sensor_data())

    motor_go(int(wl),int(wr))

    if abs(wl) > 4000 or abs(wr) > 4000:
        print(f"SATURATED")

    time.sleep(DT)
motor_stop()

44.13625 13.64 2.687290566896477
------------------------------------- [0, 1808, 0, 0, 0, 0, 0]
44.13625 13.64 2.6903283125068738
------------------------------------- [0, 1802, 0, 0, 0, 0, 0]
44.058749999999996 13.678749999999999 2.7062393784374907
------------------------------------- [0, 1802, 0, 0, 0, 0, 0]
43.43875 13.91125 2.773922541896684
------------------------------------- [0, 1996, 0, 0, 0, 0, 0]
42.74125 14.14375 2.8555412118724752
------------------------------------- [1038, 2200, 0, 0, 0, 0, 0]
42.005 14.29875 2.9373384907234126
------------------------------------- [1038, 2200, 0, 0, 0, 0, 0]
41.26875 14.45375 3.0426262481742046
------------------------------------- [1038, 2200, 0, 0, 0, 0, 0]
40.61 14.45375 -3.120319267565732
------------------------------------- [1038, 2200, 0, 0, 0, 0, 0]
39.99 14.3375 -2.999695598985629
------------------------------------- [1038, 2200, 0, 0, 0, 0, 0]
39.33125 14.22125 -2.889705637499181
------------------------------------- [0, 0, 

: 

In [ ]:
'''
get ınıtıal state from camera
get control ınput from ınıtıal state

for loop
 - get posıtıon from camera
 
 - update kalman fılter wıth data from camera

 - get posıtıon from kalman fılter

 - get control ınput from local navıgatıon controller

 - repeat

'''

In [27]:
motor_stop()